In [10]:
import json
import textstat
import pandas as pd

# Funzione per calcolare il Flesch Reading Ease (FRE)
def calculate_fre(text):
    return textstat.flesch_reading_ease(text)

# Carica i dati dal file JSON
with open('../../data/experiment_results_with_llama_and_similarity_k_10.json', 'r') as file:
    experiment_data = json.load(file)

# Lista per memorizzare i dati nel nuovo formato
formatted_data = []

# Estrai e calcola il FRE per ogni ricetta
for recipe in experiment_data:
    # Calcola il Flesch Reading Ease per l'explanation
    fre_score = calculate_fre(recipe["llama_explanation"])
    
    # Crea un dizionario per ciascuna ricetta
    recipe_info = {
        "recipe_name": recipe["recipe_name"],
        "sustainameal_recipes": [f"Recipe: {r['title']}" for r in recipe["ordered_by_sustainameal"]],
        "llama_explanation": recipe["llama_explanation"],
        "flesch_reading_ease": fre_score 
    }
    
    # Aggiungi il dizionario alla lista
    formatted_data.append(recipe_info)

# Converti i dati in un DataFrame per facilitare le operazioni
df = pd.DataFrame(formatted_data)

# Dividi il FRE in 4 gruppi (quartili)
df['readability_group'] = pd.qcut(df['flesch_reading_ease'], 4, labels=[1, 2, 3, 4])

# Converte il DataFrame di nuovo in una lista di dizionari
final_data = df.to_dict(orient='records')

# Salva i dati in un nuovo file JSON con il gruppo di leggibilità
with open('evaluation_recipes_form.json', 'w') as outfile:
    json.dump(final_data, outfile, indent=4)

# Print how many recipes are in each readability group
print(df['readability_group'].value_counts())

# Print the FRE mean for each readability group
print(df.groupby('readability_group')['flesch_reading_ease'].mean())

readability_group
2    26
1    25
4    25
3    24
Name: count, dtype: int64
readability_group
1    49.952800
2    58.988846
3    62.737917
4    70.467200
Name: flesch_reading_ease, dtype: float64


In [9]:
import json
import random

# Carica i dati dal file JSON con i gruppi di leggibilità
with open('evaluation_recipes_form.json', 'r') as file:
    form_data = json.load(file)

# Raggruppa le ricette per "readability_group"
grouped_recipes = {1: [], 2: [], 3: [], 4: []}

for recipe in form_data:
    group = recipe["readability_group"]
    grouped_recipes[group].append(recipe)

# Seleziona casualmente 5 ricette da ciascun gruppo
selected_recipes = []
for group, recipes in grouped_recipes.items():
    selected_recipes.extend(random.sample(recipes, min(5, len(recipes))))  # Prende 5 ricette per gruppo, o meno se ci sono meno di 5 ricette

# Lista per memorizzare il testo formattato per il form
form_text = []

# Introduzione al form
introduction = """Benvenuto/a! Questo modulo è progettato per valutare la qualità delle risposte generate da un sistema di raccomandazione di ricette. 
Per favore, leggi attentamente ciascuna ricetta e la sua spiegazione, e rispondi alle seguenti domande in base ai criteri specificati. 
Le tue risposte aiuteranno a migliorare il sistema e a garantire che le raccomandazioni siano sia utili che accurate."""

form_text.append(introduction)
form_text.append("\n")

# Template delle domande
evaluation_criteria = [
    {
        "criterion": "Relevance",
        "question": "Are the facts presented relevant to the question?"
    },
    {
        "criterion": "Informativeness",
        "question": "Does the response include all the information required by the question?"
    },
    {
        "criterion": "Correctness",
        "question": "How accurate is the generated response?"
    },
    {
        "criterion": "Clarity",
        "question": "Does the question ask for specific formatting, or is the response concise or verbose?"
    },
    {
        "criterion": "Hallucination",
        "question": "Does the answer include any hallucinated references or information?"
    }
]

# Genera il testo per ciascuna ricetta selezionata
for i, recipe in enumerate(selected_recipes):
    # Titolo della ricetta e informazioni aggiuntive (gruppo e FRE)
    form_text.append(f"Ricetta {i + 1}: {recipe['recipe_name']}")
    form_text.append(f"Gruppo di leggibilità: {recipe['readability_group']}")  # Aggiungi il gruppo
    form_text.append(f"Flesch Reading Ease (FRE): {recipe['flesch_reading_ease']:.2f}")  # Aggiungi il punteggio FRE
    
    # Input
    form_text.append("Input: Choose the most healthy and sustainable recipe from the list below and explain why it is the best in a user friendly paragraph, without comparing it to the others.")
    for rec in recipe["sustainameal_recipes"]:
        form_text.append(rec)
    
    # Output
    form_text.append(f"Output: {recipe['llama_explanation']}")
    
    # Aggiungi uno spazio vuoto tra una ricetta e l'altra
    form_text.append("\n")

# Salva il testo generato in un file di testo
with open('google_form_structure.txt', 'w') as outfile:
    outfile.write("\n".join(form_text))
